Problem 1:

In [ ]:
import numpy as np
def flip_coins():
    flips = np.random.randint(0, 2, 10)
    return np.mean(flips)

def experiment():
  proportions = np.random.randint(0, 2, (1000, 10)).mean(axis=1)
  first_coin = proportions[0]
  rand_coin = proportions[np.random.randint(0, 1000)]
  min_coin = proportions.min()
  return first_coin, rand_coin, min_coin

results = np.array([experiment() for _ in range(100000)])
firsts = results[:, 0]
randoms = results[:, 1]
mins = results[:, 2]
print("v min: ", np.mean(mins))
print("v rand: ", np.mean(randoms))
print("v 1: ", np.mean(firsts))

v min:  0.037748
v rand:  0.49979399999999996
v 1:  0.5003019999999999


Problem 2:

In [ ]:
#choose epsilon = 0.2, so lower bound is 0.3 and upper bound is 0.7
count1 = np.sum((firsts < 0.3) | (firsts > 0.7))
count2 = np.sum((randoms < 0.3) | (randoms > 0.7))
count3 = np.sum((mins < 0.3) | (mins > 0.7))

print("P[|v1 - u| > e]: ", count1 / 100000)
print("P[|vrand - u| > e]: ", count2 / 100000)
print("P[|vmin - u| > e]: ", count3 / 100000)

P[|v1 - u| > e]:  0.11058
P[|vrand - u| > e]:  0.1093
P[|vmin - u| > e]:  1.0


Problem 5:

In [ ]:
import random
import numpy as np

N = 100

def rand_func():
  x1 = random.uniform(-1.0, 1.0)
  x2 = random.uniform(-1.0, 1.0)
  y1 = random.uniform(-1.0, 1.0)
  y2 = random.uniform(-1.0, 1.0)
  m = (y2 - y1) / (x2 - x1)
  b = y1 - m * x1
  return m, b

def classify_point(point, m, b):
    x, y = point
    if y > m * x + b:
      return 1
    if y == m * x + b:
      return 0
    return -1

def create_points():
  m, b = rand_func()
  X = []
  Y = []
  X = np.random.uniform(-1.0, 1.0, (N, 2))
  Y = np.array([classify_point(point, m, b) for point in X])
  return [m, b], X, Y

def find_error(X, Y, w):
  Y_pred = np.sign(np.dot(X, w))
  return np.sum(Y_pred != Y) / len(Y)

f = []
g = []
errors = []
for _ in range(1000):
  [m, b], X, Y = create_points()
  f.append([m, b])
  X_with_bias = np.hstack((np.ones((X.shape[0], 1)), X))
  x_transpose = X_with_bias.T
  inv = np.linalg.inv(np.dot(x_transpose, X_with_bias))
  w = np.dot(np.dot(inv, x_transpose), Y)
  g.append(w)
  errors.append(find_error(X_with_bias, Y, w))

print(sum(errors) / len(errors))

0.03801000000000004


Problem 6:

In [ ]:
def create_test_points(m, b):
  X = np.random.uniform(-1.0, 1.0, (1000, 2))
  Y = np.array([classify_point(point, m, b) for point in X])
  return X, Y

misclassified = 0
errors_out = []
for i in range(1000):
  X_test, y_g = create_test_points(g[i][0], g[i][1])
  y_actual = np.array([classify_point(point, f[i][0], f[i][1]) for point in X_test])
  X_test_with_bias = np.hstack((np.ones((X_test.shape[0], 1)), X_test))
  errors_out.append(find_error(X_test_with_bias, y_actual, g[i]))

print(sum(errors) / len(errors))

0.03801000000000004


Problem 7:

In [ ]:
def create_PLA_points():
  m, b = rand_func()
  X = []
  Y = []
  X = np.random.uniform(-1.0, 1.0, (10, 2))
  Y = np.array([classify_point(point, m, b) for point in X])
  return X, Y

def PLA(weights, X, Y):
  iterations = 0
  while iterations < 10000:
    misclassified = []
    for i, point in enumerate(X):
      if np.sign(np.dot(weights[1:], point) + weights[0]) != Y[i]:
        misclassified.append(i)

    if len(misclassified) == 0:
      break

    idx = np.random.choice(misclassified)
    weights[0] += Y[idx]
    weights[1:] += Y[idx] * X[idx]
    iterations += 1
  return iterations

total_iterations = 0

for _ in range(1000):
  x_train, y_train = create_PLA_points()
  x_train_bias = np.hstack((np.ones((x_train.shape[0], 1)), x_train))
  x_transpose = x_train_bias.T
  inv = np.linalg.inv(np.dot(x_transpose, x_train_bias))
  w = np.dot(np.dot(inv, x_transpose), y_train)
  total_iterations += PLA(w, x_train, y_train)

print(total_iterations / 1000)

3.568


Problem 8:

In [ ]:
def classify_nonlinear(point):
  x1 = point[0]
  x2 = point[1]
  val = x1**2 + x2**2 - 0.6
  if val == 0:
    return 0
  elif val < 0:
    return -1
  return 1

def create_nonlinear_points():
  X = []
  Y = []
  X = np.random.uniform(-1.0, 1.0, (1000, 2))
  Y = np.array([classify_nonlinear(point) for point in X])
  random_indices = np.random.choice(np.arange(1000), size=100, replace=False)
  for i in random_indices:
    Y[i] *= -1
  return X, Y

nonlinear_errors = []
for _ in range(1000):
  X, Y = create_nonlinear_points()
  X_with_bias = np.hstack((np.ones((X.shape[0], 1)), X))
  x_transpose = X_with_bias.T
  inv = np.linalg.inv(np.dot(x_transpose, X_with_bias))
  w = np.dot(np.dot(inv, x_transpose), Y)
  nonlinear_errors.append(find_error(X_with_bias, Y, w))

print(sum(nonlinear_errors) / len(nonlinear_errors))

0.5036800000000006


Problem 9:

In [ ]:
def create_transform_points():
  X = []
  Y = []
  X = np.random.uniform(-1.0, 1.0, (1000, 2))
  Y = np.array([classify_nonlinear(point) for point in X])
  noise_indices = np.random.choice(len(Y), 100, replace=False)
  Y[noise_indices] = -Y[noise_indices]
  x1 = X[:, 0]
  x2 = X[:, 1]
  bias = np.ones(X.shape[0])
  X_transformed = np.column_stack((bias, x1, x2, x1 * x2, x1 ** 2, x2 ** 2))
  return X, X_transformed, Y

def check_agreement(point, h):
  x1 = point[0]
  x2 = point[1]
  return np.sign(h[0] + x1 *h[1] + x2 *h[2] + x1*x2*h[3] + h[4] * x1**2 + h[5] * x2**2)


#now we want to see over 1000 runs, average probability
h1_agree = 0
h2_agree = 0
h3_agree = 0
h4_agree = 0
h5_agree = 0

h1 = np.array([-1, -0.05, 0.08, 0.13, 1.5, 1.5])
h2 = np.array([-1, -0.05, 0.08, 0.13, 1.5, 15])
h3 = np.array([-1, -0.05, 0.08, 0.13, 15, 1.5])
h4 = np.array([-1, -1.5, 0.08, 0.13, 0.05, 0.05])
h5 = np.array([-1, -0.05, 0.08, 1.5, 0.15, 0.15])

print(w)
X_before, X, Y = create_transform_points()
x_transpose = X.T
inv = np.linalg.inv(np.dot(x_transpose, X))
w = np.dot(np.dot(inv, x_transpose), Y)

for _ in range(1000):
  x1 = np.random.uniform(-1.0, 1.0)
  x2 = np.random.uniform(-1.0, 1.0)
  w_sign =  check_agreement([x1, x2], w)
  if check_agreement([x1, x2], h1) == w_sign:
    h1_agree += 1
  if check_agreement([x1, x2], h2) == w_sign:
    h2_agree += 1
  if check_agreement([x1, x2], h3) == w_sign:
    h3_agree += 1
  if check_agreement([x1, x2], h4) == w_sign:
    h4_agree += 1
  if check_agreement([x1, x2], h5) == w_sign:
    h5_agree += 1

print("hypothesis 1: ", h1_agree/1000)
print("hypothesis 2: ", h2_agree/1000)
print("hypothesis 3: ", h3_agree/1000)
print("hypothesis 4: ", h4_agree/1000)
print("hypothesis 5: ", h5_agree/1000)

[0.06570125 0.02862171 0.03411385]
hypothesis 1:  0.948
hypothesis 2:  0.674
hypothesis 3:  0.68
hypothesis 4:  0.625
hypothesis 5:  0.545


Problem 10:

In [ ]:
def generate_with_noise():
  X = []
  Y = []
  X = np.random.uniform(-1.0, 1.0, (1000, 2))
  Y = np.array([classify_nonlinear(point) for point in X])
  noise_indices = np.random.choice(len(Y), 100, replace=False)
  Y[noise_indices] = -Y[noise_indices]
  x1 = X[:, 0]
  x2 = X[:, 1]
  bias = np.ones(X.shape[0])
  X_transformed = np.column_stack((bias, x1, x2, x1 * x2, x1 ** 2, x2 ** 2))
  return X_transformed, Y

error_total = 0

for _ in range(1000):
  x_trans, y_noise = generate_with_noise()
  error_total += find_error(x_trans, y_noise, h1)

print(error_total / 1000)

0.14300700000000027
